In [1]:
% pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.io.wavfile import read as sciread
from scipy import stats

Populating the interactive namespace from numpy and matplotlib


In [2]:
def calcFFT(audio, fs):
    transformed = np.fft.fft(audio)
    f = np.arange(0, len(transformed))*fs/len(transformed)
    f = f[:44100]
    transformed = transformed[:44100]
    return (f, transformed)

In [3]:
def plotFFT(audio, fs):
    f, transformed = calcFFT(audio, fs)
    plt.plot(f, np.abs(transformed))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.show()

In [4]:
def get_tempo(signal, sr):
    onset_env = librosa.onset.onset_strength(signal, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    return tempo

In [5]:
def get_harmonic(signal, sr):
    signal = librosa.effects.harmonic(signal)
    f, transformed = calcFFT(signal, sr)
    transformed = np.array(np.abs(transformed))
    data = {}
    data['hmedian'] = np.median(transformed)
    data['hmean'] = np.mean(transformed)
    data['h1Q'] = np.percentile(transformed, 25)
    data['h2Q'] = np.percentile(transformed, 75)
    data['hIQR'] = stats.iqr(transformed)
    data['hmin'] = np.min(transformed)
    data['hmax'] = np.max(transformed)
    data['hstd'] = np.std(transformed)
    data['htempo'] = get_tempo(signal, 44100)
    return data

In [6]:
def get_percussion(signal, sr):
    signal = librosa.effects.percussive(signal)
    f, transformed = calcFFT(signal, sr)
    transformed = np.array(np.abs(transformed))
    data = {}
    data['pmedian'] = np.median(transformed)
    data['pmean'] = np.mean(transformed)
    data['p1Q'] = np.percentile(transformed, 25)
    data['p2Q'] = np.percentile(transformed, 75)
    data['pIQR'] = stats.iqr(transformed)
    data['pmin'] = np.min(transformed)
    data['pmax'] = np.max(transformed)
    data['pstd'] = np.std(transformed)
    data['ptempo'] = get_tempo(signal, 44100)
    return data

In [ ]:
def get_mfcc(signal, sr):
    S = librosa.feature.melspectrogram(y=signal, sr=sr, n_mels=128, fmax=8000)
    signal = librosa.feature.mfcc(S=librosa.power_to_db(S))
    data = {}
    data['M_median'] = np.median(signal)
    data['M_mean'] = np.mean(signal)
    data['M_1Q'] = np.percentile(signal, 25)
    data['M_2Q'] = np.percentile(signal, 75)
    data['M_IQR'] = stats.iqr(signal)
    data['M_min'] = np.min(signal)
    data['M_max'] = np.max(signal)
    data['M_std'] = np.std(signal)
    return data

In [7]:
def get_info(fpath, label):
    signal = librosa.core.load(fpath, 44100)[0]
    f, transformed = calcFFT(signal, 44100)
    transformed = np.array(np.abs(transformed))
    harmonic_data = get_harmonic(signal, 44100)
    percussion_data = get_percussion(signal, 44100)
    mfcc_data = get_mfcc(signal, 44100)
    data = {}
    data['fpath'] = fpath
    data['label'] = label
    data['median'] = np.median(transformed)
    data['mean'] = np.mean(transformed)
    data['1Q'] = np.percentile(transformed, 25)
    data['2Q'] = np.percentile(transformed, 75)
    data['IQR'] = stats.iqr(transformed)
    data['min'] = np.min(transformed)
    data['max'] = np.max(transformed)
    data['std'] = np.std(transformed)
    data['tempo'] = get_tempo(signal, 44100)
    data = {**data, **percussion_data, **harmonic_data, **mfcc_data}
    return data

In [8]:
train = pd.read_csv('train.csv')
train.head(10)

,Unnamed: 0,fname,label,manually_verified,fpath
0,0,00044347.wav,Hi-hat,0,audio_train/audio_train/00044347.wav
1,1,001ca53d.wav,Saxophone,1,audio_train/audio_train/001ca53d.wav
2,2,002d256b.wav,Trumpet,0,audio_train/audio_train/002d256b.wav
3,3,0033e230.wav,Glockenspiel,1,audio_train/audio_train/0033e230.wav
4,4,00353774.wav,Cello,1,audio_train/audio_train/00353774.wav
5,5,003b91e8.wav,Cello,0,audio_train/audio_train/003b91e8.wav
6,6,003da8e5.wav,Knock,1,audio_train/audio_train/003da8e5.wav
7,7,0048fd00.wav,Gunshot_or_gunfire,1,audio_train/audio_train/0048fd00.wav
8,8,004ad66f.wav,Clarinet,0,audio_train/audio_train/004ad66f.wav
9,9,0063ab88.wav,Computer_keyboard,0,audio_train/audio_train/0063ab88.wav


In [12]:
working = pd.read_csv('working.csv')

In [17]:
import time

In [20]:
initial_time = time.time()
for i in range(working.index.size, train.fname.size):
    fpath = train.loc[i, 'fpath']
    label = train.loc[i, 'label']
    info = get_info(fpath, label)
    df = pd.DataFrame(info, index=[i])
    working = pd.concat([working, df])
    if i % 100 == 0:
        new_time = time.time()
        time_difference = new_time - initial_time
        initial_time = new_time
        print('Iteration: {}. Time for the last 100 Iterations: {} seconds'.format(i, time_difference))

Iteration: 3000. Time for the last 100 Iterations: 314.32115936279297 seconds
Iteration: 3100. Time for the last 100 Iterations: 343.1671600341797 seconds
Iteration: 3200. Time for the last 100 Iterations: 327.1750257015228 seconds
Iteration: 3300. Time for the last 100 Iterations: 329.1874897480011 seconds
Iteration: 3400. Time for the last 100 Iterations: 315.18456959724426 seconds
Iteration: 3500. Time for the last 100 Iterations: 268.665052652359 seconds
Iteration: 3600. Time for the last 100 Iterations: 300.31506395339966 seconds
Iteration: 3700. Time for the last 100 Iterations: 319.07259941101074 seconds
Iteration: 3800. Time for the last 100 Iterations: 322.8154993057251 seconds
Iteration: 3900. Time for the last 100 Iterations: 316.18946146965027 seconds
Iteration: 4000. Time for the last 100 Iterations: 346.87124824523926 seconds
Iteration: 4100. Time for the last 100 Iterations: 274.25691843032837 seconds
Iteration: 4200. Time for the last 100 Iterations: 384.58490014076233 

In [23]:
working.to_csv(path_or_buf='working.csv')